In [1]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define dataset path (update with your actual path)
dataset_path = "/content/drive/MyDrive/Stenosis detection/arcade"
os.listdir(dataset_path)  # List dataset files


Mounted at /content/drive


['stenosis',
 'syntax',
 'weights',
 'weights_CBAM+optimization+noaug',
 'dataset2.yaml',
 'train_masks',
 'val_masks',
 'train_images',
 'weights_yolov9+CBAM+SimCLR',
 'test_masks',
 'val_images',
 'efficientnet_stenosis.keras',
 'resnet_stenosis.keras',
 'train',
 'val',
 'test',
 'dataset3.yaml',
 'dataset.yaml',
 'model.pt',
 'mobilenet_stenosis.keras',
 'vgg_stenosis.keras',
 'inceptionV3_stenosis.keras']

In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [3]:
from ultralytics import YOLO

# Load trained YOLOv9 model with best weights
yolo_model = YOLO("/content/drive/MyDrive/Stenosis detection/arcade/weights_yolov9+CBAM+SimCLR/best.pt")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
import json
import numpy as np

def load_ground_truth(json_path):
    """
    Loads ground truth bounding boxes from the dataset JSON file.
    """
    with open(json_path, 'r') as f:
        data = json.load(f)

    ground_truth = {}

    for image in data['images']:
        img_id = image['id']
        img_filename = image['file_name']

        # Get bounding boxes for this image
        bboxes = []
        for ann in data['annotations']:
            if ann['image_id'] == img_id:
                bbox = ann['bbox']  # COCO format: [xmin, ymin, width, height]
                xmin, ymin, w, h = bbox
                xmax, ymax = xmin + w, ymin + h
                bboxes.append([xmin, ymin, xmax, ymax, 1])  # Class "stenosis" = 1

        ground_truth[img_filename] = np.array(bboxes)  # Store as numpy array

    return ground_truth


In [5]:
json_file=f"{dataset_path}/stenosis/test/annotations/test.json"
gt= load_ground_truth(json_file)

In [6]:
def yolo_detect(image_path):
   results = yolo_model(image_path)  # Run inference
   # Extract bounding boxes from YOLO's output
   detected_boxes = []
   for result in results:
    for box in result.boxes.data:
      xmin, ymin, xmax, ymax, confidence, class_id = box.cpu().numpy()
      detected_boxes.append([xmin, ymin, xmax, ymax, confidence, int(class_id)])
   return {image_path: detected_boxes} # List of detected boxes

In [7]:
from tensorflow.keras.models import load_model

# Load Trained EfficientNet Model
inceptionv3_model = load_model("/content/drive/MyDrive/Stenosis detection/arcade/inceptionV3_stenosis.keras")


In [8]:
import cv2
def predict_stenosis(image_path, threshold=0.2):
    """
    Runs Stenosis Classification on a Full Angiogram Image.
    """
    image = cv2.imread(image_path)
    image_resized = cv2.resize(image, (512, 512)) / 255.0  # Normalize
    image_input = np.expand_dims(image_resized, axis=0)  # Add batch dimension

    prediction = inceptionv3_model.predict(image_input)[0][0]  # Get probability

    if prediction > 0.5:
        return 1  # Stenosis Detected
    else:
        return 0  # No Stenosis


In [9]:
def detect_stenosis(image_path):

    # Step 1: Classify the Full Image Using EfficientNet
    stenosis_detected = predict_stenosis(image_path)

    # Step 2: If No Stenosis, Return No Detections
    if stenosis_detected == 0:
        return []

    # Step 3: Otherwise, Run YOLOv9 for Object Detection
    results = yolo_model(image_path)
    final_detections = []

    for result in results:
        for box in result.boxes.data:
            xmin, ymin, xmax, ymax, confidence, class_id = box.cpu().numpy()

            # Keep only Stenosis (Class 0) Detections
            if class_id == 0:
                final_detections.append([xmin, ymin, xmax, ymax, confidence])

    return final_detections


In [10]:
image_path = "/content/drive/MyDrive/Stenosis detection/arcade/stenosis/test/images/1.png"


In [ ]:
import cv2
from google.colab.patches import cv2_imshow
detections= detect_stenosis(image_path)
def visualize_detections(image_path, detections):

    image = cv2.imread(image_path)

    for (xmin, ymin, xmax, ymax, confidence) in detections:
        cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2)
        cv2.putText(image, f"Stenosis ({confidence:.2f})", (int(xmin), int(ymin) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2_imshow( image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [11]:
import os

def run_combined_model_on_dataset(test_folder, gt_boxes):

    combined_detections = {}

    for filename in os.listdir(test_folder):
        if filename.endswith(".png") or filename.endswith(".jpg"):
            image_path = os.path.join(test_folder, filename)
            detections = detect_stenosis(image_path)

            combined_detections[filename] = detections

    return combined_detections


In [ ]:

test_folder = "/content/drive/MyDrive/Stenosis detection/arcade/stenosis/test/images/"
final_detections = run_combined_model_on_dataset(test_folder, gt)


In [12]:
import os

# Path to the test images folder
test_folder = "/content/drive/MyDrive/Stenosis detection/arcade/stenosis/test/images/"

# Create a dictionary where each image has a label 1 (Stenosis Present)
gt_labels = {filename: 1 for filename in os.listdir(test_folder) if filename.endswith((".png", ".jpg"))}



In [13]:
from sklearn.metrics import precision_score, recall_score, f1_score


predictions = []
true_labels = []

for filename, label in gt_labels.items():
    image_path = test_folder + filename
    pred = 1 if detect_stenosis(image_path) else 0  # If YOLO detects anything, classify as 1

    predictions.append(pred)
    true_labels.append(label)



1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 848ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 960ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step

image 1/1 /content/drive/MyDrive/Stenosis detection/arcade/stenosis/test/images/135.png: 512x512 1 stenosis, 2125.7ms
Speed: 8.2ms preprocess, 2125.7ms inference, 24.4ms postprocess per image at shape (1, 3, 512, 512)
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 890ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 886ms/step

image 1/1 /content/drive/MyDrive/Stenosis detection/arcade/stenosis/test/images/109.png: 512x512 2 stenosiss, 436.5ms
Speed: 2.0ms preprocess, 436.5ms inference, 0.9ms postprocess per image at shape (1, 3, 512, 512)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 882ms/step

image 1/1 /content/drive/MyDrive/Stenosis de

In [14]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Compute Metrics
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)
accuracy = accuracy_score(true_labels, predictions)


print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}, Accuracy: {accuracy:.4f}")

Precision: 1.0000, Recall: 0.1900, F1-score: 0.3193, Accuracy: 0.1900
